In [49]:
# VERİ SETİNİ GETİRME
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

data = sns.load_dataset("tips")
df = pd.DataFrame(data)

# Eksik Verileri Ortalama ile Doldurma
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputer = imputer.fit(df[["total_bill"]])
df[["total_bill"]] = imputer.transform(df[["total_bill"]])

# Kategorik Olmayan Değişkenler
total_bill = df[["total_bill"]]
tip = df[["tip"]]
size = df[["size"]]

# Ktegorik Değişkenler İçin 0-1 Dönüşümü
smoker = pd.get_dummies(df[["smoker"]])
time = pd.get_dummies(df[["day"]])

# Kukla Değişkeni Silme
smoker = smoker.drop("smoker_Yes", axis=1)

# Verileri Birleştirme
new_data = pd.concat([total_bill, size, smoker, time, tip], axis=1)

# Train ve Test Belirleme
training = new_data.iloc[:, 0:7]
testing = new_data.iloc[:, 7:8]

# Train Test Ayırma
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(training, testing, test_size=0.30, random_state=42)

In [50]:
import warnings
warnings.filterwarnings('ignore')

In [51]:
from sklearn.svm import SVR

In [52]:
svr_model = SVR()
svr_model.fit(x_train, y_train)

SVR()

In [53]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = svr_model.predict(x_test)

r2 = r2_score(y_test, y_pred)

mse = np.sqrt(mean_squared_error(y_test, y_pred))

print("----------------------")
print("İlk Ortalama Karekök: ", mse)
print("----------------------")
print("İlk R2 Değeri: ", r2)

----------------------
İlk Ortalama Karekök:  0.8582732573436449
----------------------
İlk R2 Değeri:  0.4411781022689396


### Model Tunning

In [54]:
svr_params = {"C": np.arange(0.1, 10, 0.1),
             "degree":np.arange(1,5,1),
             "kernel":["linear", "poly", "rbf", "sigmoid"]}

In [55]:
from sklearn.model_selection import GridSearchCV

svr_cv_model = GridSearchCV(svr_model, svr_params, cv=10)
svr_cv_model.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=SVR(),
             param_grid={'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9]),
                         'degree': array([1, 2, 3, 4]),
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

In [56]:
print(svr_cv_model.best_params_)

{'C': 8.6, 'degree': 1, 'kernel': 'rbf'}


In [57]:
svr_tuned = SVR(C = 8.6, degree=1, kernel='rbf')
svr_tuned.fit(x_train, y_train)

SVR(C=8.6, degree=1)

### MSE v R2 Score

MSE --> Tahmin edilen değerler gerçek değerlere ne kadar yakınsa MSE o kadar küçük olur; gerçek değerlerden ne kadar uzaklaşırsa MSE o kadar büyük olur. 

r2 --> R2‘ın yüksek olması regresyon model uyumunun iyi olduğunu gösterir.

In [58]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred=svr_tuned.predict(x_test)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("----------------------")
print("İkinci Ortalama Karekök: ", rmse)
print("----------------------")
print("İkinci R2 Değeri: ", r2)

----------------------
İkinci Ortalama Karekök:  0.9053257760117702
----------------------
İkinci R2 Değeri:  0.3782267618084061
